In [1]:
import pandas as pd
import datetime
import numpy as np
import pickle
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
# np.seed(42)

In [2]:
def filter_by_recent_date(df, date_col):
    # get most recent date by site code
    df_date_only_recent = df.sort_values(date_col).groupby('SiteCode').tail(1)
    most_recent_dates = df_date_only_recent[['SiteCode',date_col]]
    # make a look up dictionary for match_site_code_date
    most_recent_dates_dict = dict(zip(most_recent_dates['SiteCode'], most_recent_dates[date_col]))
    # get all rows for the most recent date.
    date_idxs = match_site_code_date(date_col, df, most_recent_dates_dict)
    df = df[df.index.isin(date_idxs)]  
    df = df.rename(columns={date_col: 'Date'})
    
    if date_col == 'StartDate':
        df = df.drop(columns=['EndDate'])
    else:
        df = df.drop(columns=['StartDate'])
    #     df['isStartDate'] = False

    return df

def match_site_code_date(date_col, df, recent_dict):
    match_idx = []
    
    for idx, row in df.iterrows():
        if row['SiteCode'] in recent_dict and row[date_col] == recent_dict[row['SiteCode']]:
            match_idx.append(idx)
            
    return match_idx

def create_recent_mindex(df):
    ni_site_codes = df[df['CommodityName'] == 'Nickel']['SiteCode']
    df = df[df['SiteCode'].isin(ni_site_codes)]

    df_no_edate = df[df['EndDate'].isna()]
    df_edate = df[~df['EndDate'].isna()]

    df_no_edate = filter_by_recent_date(df_no_edate, date_col='StartDate')
    df_edate = filter_by_recent_date(df_edate, date_col='EndDate')

    df_both = pd.concat([df_edate, df_no_edate], axis=0)

    return df_both

In [3]:
mindex = pd.read_csv('../../hank/Transformed_Minedex_data.csv')
recent_mindex = create_recent_mindex(mindex)

In [4]:
recent_mindex.to_csv('../../data/recent_mindex.csv')

In [5]:
##test

In [25]:
full_to_atomic = {'Hydrogen' : 'H', 'Helium' : 'He', 'Lithium' : 'Li', 'Beryllium' : 'Be', 'Boron' : 'B', 'Carbon' : 'C', 'Nitrogen' : 'N', 'Oxygen' : 'O', 'Fluorine' : 'F', 'Neon' : 'Ne', 'Sodium' : 'Na', 'Magnesium' : 'Mg', 'Aluminium' : 'Al', 'Silicon' : 'Si', 'Phosphorus' : 'P', 'Sulfur' : 'S', 'Chlorine' : 'Cl', 'Argon' : 'Ar', 'Potassium' : 'K', 'Calcium' : 'Ca', 'Scandium' : 'Sc', 'Titanium' : 'Ti', 'Vanadium' : 'V', 'Chromium' : 'Cr', 'Manganese' : 'Mn', 'Iron' : 'Fe', 'Cobalt' : 'Co', 'Nickel' : 'Ni', 'Copper' : 'Cu', 'Zinc' : 'Zn', 'Gallium' : 'Ga', 'Germanium' : 'Ge', 'Arsenic' : 'As', 'Selenium' : 'Se', 'Bromine' : 'Br', 'Krypton' : 'Kr', 'Rubidium' : 'Rb', 'Strontium' : 'Sr', 'Yttrium' : 'Y', 'Zirconium' : 'Zr', 'Niobium' : 'Nb', 'Molybdenum' : 'Mo', 'Technetium' : 'Tc', 'Ruthenium' : 'Ru', 'Rhodium' : 'Rh', 'Palladium' : 'Pd', 'Silver' : 'Ag', 'Cadmium' : 'Cd', 'Indium' : 'In', 'Tin' : 'Sn', 'Antimony' : 'Sb', 'Tellurium' : 'Te', 'Iodine' : 'I', 'Xenon' : 'Xe', 'Cesium' : 'Cs', 'Barium' : 'Ba', 'Lanthanum' : 'La', 'Cerium' : 'Ce', 'Praseodymium' : 'Pr', 'Neodymium' : 'Nd', 'Promethium' : 'Pm', 'Samarium' : 'Sm', 'Europium' : 'Eu', 'Gadolinium' : 'Gd', 'Terbium' : 'Tb', 'Dysprosium' : 'Dy', 'Holmium' : 'Ho', 'Erbium' : 'Er', 'Thulium' : 'Tm', 'Ytterbium' : 'Yb', 'Lutetium' : 'Lu', 'Hafnium' : 'Hf', 'Tantalum' : 'Ta', 'Tungsten' : 'W', 'Rhenium' : 'Re', 'Osmium' : 'Os', 'Iridium' : 'Ir', 'Platinum' : 'Pt', 'Gold' : 'Au', 'Mercury' : 'Hg', 'Thallium' : 'Tl', 'Lead' : 'Pb', 'Bismuth' : 'Bi', 'Polonium' : 'Po', 'Astatine' : 'At', 'Radon' : 'Rn', 'Francium' : 'Fr', 'Radium' : 'Ra', 'Actinium' : 'Ac', 'Thorium' : 'Th', 'Protactinium' : 'Pa', 'Uranium' : 'U', 'Neptunium' : 'Np', 'Plutonium' : 'Pu', 'Americium' : 'Am', 'Curium' : 'Cm', 'Berkelium' : 'Bk', 'Californium' : 'Cf', 'Einsteinium' : 'Es', 'Fermium' : 'Fm', 'Mendelevium' : 'Md', 'Nobelium' : 'No', 'Lawrencium' : 'Lr', 'Rutherfordium' : 'Rf', 'Dubnium' : 'Db', 'Seaborgium' : 'Sg', 'Bohrium' : 'Bh', 'Hassium' : 'Hs', 'Meitnerium' : 'Mt', 'Darmstadtium' : 'Ds', 'Roentgenium' : 'Rg', 'Copernicium' : 'Cn', 'Nihonium' : 'Nh', 'Flerovium' : 'Fl', 'Moscovium' : 'Mc', 'Livermorium' : 'Lv', 'Tennessine' : 'Ts', 'Og' : 'Oganesson',}

def convert_to_atomic(col):
    ### NEED TO CATER TO EDGE CASES
    ### e.g Mag Oxide, Alumina etc.
    ###
    full_name = col
    
    if full_name in full_to_atomic.keys():
        atomic = full_to_atomic[full_name]
        return atomic

    else:
        return np.nan
        

recent_mindex['CommodityName'] = recent_mindex['CommodityName'].apply(convert_to_atomic)

In [26]:
### DROPPING WIERD COMMODS FOR NOW.
recent_mindex = recent_mindex[~recent_mindex['CommodityName'].isna()]

In [27]:
# get list of unique mineral combinations
list_combinations = recent_mindex.groupby('SiteCode')['CommodityName'].unique().to_list()

# sort alphabetically
list_combinations = [sorted(commods) for commods in list_combinations]

unique_commods = []
for commods in list_combinations:
    if commods not in unique_commods:
        unique_commods.append(commods)

In [28]:
unique_commods

[['Ni'],
 ['As', 'Ni'],
 ['Al', 'Ca', 'Co', 'Cu', 'Fe', 'Mg', 'Ni', 'Zn'],
 ['Cu', 'Ni'],
 ['As', 'Co', 'Cu', 'Ni', 'S'],
 ['Co', 'Ni'],
 ['Co', 'Cu', 'Ni'],
 ['Au', 'Cu', 'Ni', 'Pd', 'Pt'],
 ['Au', 'Cu', 'Ni', 'Pd', 'Pt', 'Rh'],
 ['Al', 'Co', 'Fe', 'Mg', 'Ni'],
 ['Co', 'Cu', 'Ni', 'Pd'],
 ['Co', 'Ni', 'Sc'],
 ['Al', 'Ca', 'Co', 'Fe', 'Mg', 'Ni'],
 ['Ag', 'Au', 'Co', 'Cu', 'Ni', 'Pd', 'Pt'],
 ['Co', 'Mg', 'Ni'],
 ['Co', 'Cu', 'Ni', 'S'],
 ['Co', 'Cu', 'Ni', 'Pd', 'Pt'],
 ['As', 'Co', 'Cu', 'Fe', 'Mg', 'Ni', 'S'],
 ['Co', 'Mn', 'Ni', 'Pd', 'Pt', 'Sc'],
 ['Al', 'Co', 'Fe', 'Mg', 'Mn', 'Ni'],
 ['Cu', 'Ni', 'Pd', 'Pt', 'S'],
 ['Co', 'Mn', 'Ni'],
 ['Co', 'Cu', 'Fe', 'Ni', 'S'],
 ['Cr', 'Fe', 'Ni'],
 ['Co', 'Cr', 'Fe', 'Mn', 'Ni'],
 ['Ag', 'As', 'Au', 'Cu', 'Ni'],
 ['Co', 'Cr', 'Fe', 'Ni']]

In [29]:
def replace_neg9999(val):
    if val == -9999: 
        return 0
    else:
        return val
    
    
def abs_halve_neg_val(val):
    if val < 0: 
        return abs(val) / 2
    else: 
        return val

def clean_assays(df):
    df = df.applymap(replace_neg9999)
    df = df.applymap(abs_halve_neg_val)

    # all 9999 -> 0 values are removed
    indexs_0 = df[df.values == 0].index
    df = df.drop(index=indexs_0)
    
    return df

def abundant_ratio(row, abundant_mineral):
    return row / row[abundant_mineral]

def make_pickle_fn(df, model, prefix='../../pickles/'):
    fn = f'{prefix}{type(model).__name__}-'
    for i in df.columns:
        if 'Co' in i: continue
        fn = fn + i + '-'

    fn = fn[:-1] + '.pickle'   
    return fn

def filter_by_wamex_minerals(unique_commods, wamex_unique_minerals):
    # filter unique commods by availability in wamex assays
    filtered_unique_commods = []
    for commods in unique_commods:
        filtered_commods = []
        for commod in commods:
            if commod in wamex_unique_minerals:
                filtered_commods.append(commod)
        filtered_unique_commods.append(filtered_commods)
    return filtered_unique_commods

In [30]:
# WAMEX
wamex = pd.read_csv('../../asher/Merged_Ni_lessthan15km.csv',index_col=0)
# wamex.head()

### INSERT LESS THAN KM

In [31]:
wamex = wamex.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y', 'lat', 'long', 
                     'minedex_coords', 'minedex_site',
                    'distance'])

wamex = wamex.rename(columns={
    "wamex_lat": "lat",
    "wamex_lon": "lon"
})

wamex = wamex.drop_duplicates()

In [32]:
wamex_minerals = wamex.drop(columns=['lat', 'lon'])

wamex.loc[:, wamex_minerals.columns] = clean_assays(wamex.loc[:,wamex_minerals.columns])

# Ratio transform
# wamex.loc[:, wamex_minerals.columns] = wamex.loc[:, wamex_minerals.columns].apply(
#                                                             abundant_ratio, abundant_mineral='Ni',
#                                                             axis=1)
# apply Log transform
wamex.loc[:, wamex_minerals.columns] = wamex.loc[:,wamex_minerals.columns].apply(np.log, axis=1)


# Available minerals in WAMEX
wamex_unique_minerals = wamex_minerals.columns

filtered_unique_commods = filter_by_wamex_minerals(unique_commods, wamex_unique_minerals)

In [33]:
def make_df_from_commods(unique_commods):
    if 'Co' not in unique_commods:
        unique_commods = unique_commods + ['Co']

    df_inputs = wamex[unique_commods + ['lat', 'lon']]
#     df_inputs = df_inputs.drop(columns=['Ni'])

    df_inputs = df_inputs.dropna()
    ### Needs validation!
    ###
    df_inputs = df_inputs[df_inputs['Co'] < 5000]
    
    return df_inputs

In [34]:
def commod_models(df_inputs):
    y = df_inputs['Co']
    X = df_inputs.drop(columns=['Co'])

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, shuffle=True)

    lr = LinearRegression()
    lr.fit(X_train, y_train)

    lr_fn = make_pickle_fn(df_inputs, lr)

    with open(lr_fn, 'wb') as file:
        pickle.dump(lr, file)

    train_preds = lr.predict(X_train)
    test_preds = lr.predict(X_test)

    lr_train_mse = mean_squared_error(train_preds, y_train)
    lr_test_mse = mean_squared_error(test_preds, y_test)

    # Random Forest
    rf = RandomForestRegressor(max_features=0.5)
    rf.fit(X_train, y_train)

    rf_fn = make_pickle_fn(df_inputs, rf)
    with open(rf_fn, 'wb') as file:
        pickle.dump(rf, file)

    train_preds = rf.predict(X_train)
    test_preds = rf.predict(X_test)

    rf_train_mse = mean_squared_error(train_preds, y_train)
    rf_test_mse = mean_squared_error(test_preds, y_test)
    
    return {
        "lr_path": lr_fn,
        "lr_train_mse": lr_train_mse,
        "lr_test_mse": lr_test_mse,
        "rf_train_mse": rf_train_mse,
        "rf_test_mse": rf_test_mse,
    }

In [35]:
model_outputs = []

# sample = filtered_unique_commods[:3]

for unique_commods in tqdm(filtered_unique_commods):
    df_inputs = make_df_from_commods(unique_commods)
    model_output = commod_models(df_inputs)
    model_outputs.append(model_output)

100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [00:43<00:00,  1.63s/it]


In [36]:
model_outputs_df = pd.DataFrame(model_outputs)

In [37]:
model_outputs_df.to_csv('../../data/model_errors.csv')

In [38]:
model_outputs_df

,lr_path,lr_train_mse,lr_test_mse,rf_train_mse,rf_test_mse
0,../../pickles/LinearRegression-Ni-lat-lon.pickle,1.002014,0.982939,0.056555,0.400085
1,../../pickles/LinearRegression-Ni-lat-lon.pickle,1.002014,0.982939,0.058101,0.407589
2,../../pickles/LinearRegression-Cu-Fe-Ni-lat-lo...,0.907761,0.931356,0.049900,0.349476
3,../../pickles/LinearRegression-Cu-Ni-lat-lon.p...,0.951613,0.866928,0.054427,0.375356
4,../../pickles/LinearRegression-Cu-Ni-S-lat-lon...,1.034794,0.915617,0.056519,0.399656
5,../../pickles/LinearRegression-Ni-lat-lon.pickle,1.002014,0.982939,0.057872,0.407085
6,../../pickles/LinearRegression-Cu-Ni-lat-lon.p...,0.951613,0.866928,0.054581,0.376302
7,../../pickles/LinearRegression-Au-Cu-Ni-Pd-Pt-...,0.524660,0.534778,0.043369,0.313963
8,../../pickles/LinearRegression-Au-Cu-Ni-Pd-Pt-...,0.556201,0.418707,0.072874,0.426041
9,../../pickles/LinearRegression-Fe-Ni-lat-lon.p...,0.940294,0.997591,0.050804,0.390501


In [39]:
def clean_path_string(row):
    path = row['lr_path']
    row['lr_path'] = path.split('/')[-1].split('LinearRegression')[1].split('.')[0]
    return row
    
model_outputs_df = model_outputs_df.apply(clean_path_string, axis=1)

model_outputs_df.rename(columns={"lr_path": "commodities_used"})

,commodities_used,lr_train_mse,lr_test_mse,rf_train_mse,rf_test_mse
0,-Ni-lat-lon,1.002014,0.982939,0.056555,0.400085
1,-Ni-lat-lon,1.002014,0.982939,0.058101,0.407589
2,-Cu-Fe-Ni-lat-lon,0.907761,0.931356,0.049900,0.349476
3,-Cu-Ni-lat-lon,0.951613,0.866928,0.054427,0.375356
4,-Cu-Ni-S-lat-lon,1.034794,0.915617,0.056519,0.399656
5,-Ni-lat-lon,1.002014,0.982939,0.057872,0.407085
6,-Cu-Ni-lat-lon,0.951613,0.866928,0.054581,0.376302
7,-Au-Cu-Ni-Pd-Pt-lat-lon,0.524660,0.534778,0.043369,0.313963
8,-Au-Cu-Ni-Pd-Pt-Rh-lat-lon,0.556201,0.418707,0.072874,0.426041
9,-Fe-Ni-lat-lon,0.940294,0.997591,0.050804,0.390501
